# Railway Maintenance Scheduling Problem

In [1]:
import numpy as np
import gurobipy as gb
from gurobipy import Model, quicksum
import random as random
from collections import deque

print("Gurobi version:", gb.gurobi.version())

Gurobi version: (12, 0, 0)


***


### Sets and Constants

In [ ]:
# Scalar constants
n = 10 # Number of stations
Tend = 30 # Time horizon
jobs = 5 # Number of jobs


# Set of discrete time periods													(T)
T = range(Tend)


# Set of stations																(N)
N = range(n)


# Set of arcs																	(A)
A = [(i, j) for i in N for j in N if i != j]


# Sets of connected arcs for each job											(A_j)
Aj = []
for j in range(jobs):
	start = np.random.randint(0, n) # Start station
	length = np.random.randint(1, n) # Length of the arcs for the job
	Ajob = [(start, (start + i) % n) for i in range(length)] # Arcs for the job
	Aj.append(Ajob) # Add the arcs to the list:


# Processing times for each job													(pi)
pi = np.random.randint(1, Tend//2, jobs)


# Set of Jobs																	(J)
# J = {j: (Aj[j],pi[j]) for j in range(jobs)}
J = range(jobs)


# Set of jobs on arc a															(J_a)
Ja = {a: [] for a in A}

for a in A:
	# for job in J:
		# if a in J[job][0]:
			# Ja[a].append((job, J[job][1]))
			   
	for job in J:
		if a in Aj[job]:
			Ja[a].append(job)


# Number of routes considered when travelling from any o to d					(K)
K = 3 

# Set of routes from o to d														(R)
R = {(o, d): [] for o in N for d in N if o != d}


# Function to find all paths from o to d using BFS
def find_all_paths(o, d, A):
	queue = deque([(o, [o])])
	paths = []
	while queue:
		(node, path) = queue.popleft()
		for (i, j) in A:
			if i == node and j not in path:
				new_path = path + [j]
				if j == d:
					paths.append(new_path)
				else:
					queue.append((j, new_path))
	return paths


# Generate routes for each pair (o, d)
for o in N:
	for d in N:
		if o != d:
			all_paths = find_all_paths(o, d, A)
			if len(all_paths) >= K:
				selected_paths = random.sample(all_paths, K)
			else:
				selected_paths = all_paths
			R[(o, d)] = [[(path[i], path[i+1]) for i in range(len(path)-1)] for path in selected_paths]


# Expected travel times by train on each arc									(omega^e)
omega_e = {(i, j): np.random.randint(1, 10) for (i, j) in A}


# Expected travel times by alternative services	on each arc						(omega^j)
omega_j = {(i, j): omega_e[i, j] + np.random.randint(1, 5) for (i, j) in A} 


# Passeger demand for each possible (origin, destination) pair at time t		(phi)
phi = {(i, j, t): np.random.randint(1, 100) for (i, j) in A for t in T}


# Share of daily passenger demand in peak moment in time t						(beta)
beta = {(i, j, t): np.random.uniform(0.1, 0.9) for (i, j) in A for t in T}


# Average travel time between o and d
Omega = {(o, d): 0 for o in N for d in N}
for o in N:
	for d in N:
		if o != d:
			Omega[(o, d)] = np.random.randint(1, 10)
	

# Set of arcs that cannot be unavailable simultaneously							(C)
C = [
	[A[0], A[5], A[27]],
	[A[1], A[6], A[28], A[3]],
	[A[80], A[9], A[29], A[4]],
	[A[2], A[17]],
]


# Limited capacity of alternative services in segment s in A at time t			(gamma)
gamma = 80


# Minimum time interval in days between maintanance of arc a					(tau)
tau = {a: np.random.randint(1, 5) for a in A}


# Set of event tracks at each time t											(Et)
Et = {}
for t in T:
	n_event = np.random.randint(1, 10) # np.random number of event tracks
	Et_tmp = []

	# np.randomly sample n_event arcs from A
	for i in range(n_event):
		n_rand = np.random.randint(0, len(A)-1)
		Et_tmp.append(A[n_rand])

	Et[t] = Et_tmp


***

### Model

In [3]:
# Model
model = Model()
model.ModelSense = gb.GRB.MINIMIZE

Set parameter Username
Set parameter LicenseID to value 2585388
Academic license - for non-commercial use only - expires 2025-11-15


***

### Decision Variables

In [4]:
# Start time of j^th job														(y)
# y[j,t] = 1 if job j starts at time t, 0 otherwise
# y = model.addVars(len(J), len(T), vtype=gb.GRB.BINARY, name='y')
y = model.addVars(J, T, vtype=gb.GRB.BINARY, name='y')

# Availability of arc a at time t												(x)
# x[a,t] = 1 if arc a is available at time t, 0 otherwise
# x = model.addVars(len(A), len(T), vtype=gb.GRB.BINARY, name='x')
x = model.addVars(A, T, vtype=gb.GRB.BINARY, name='x')

# Route option k used when travelling from o to d at time t						(h)
# h[o,d,t,k] = 1 if route option k is used when travelling from o to d at time t, 0 otherwise
# h = model.addVars(len(N), len(N), len(T), K, vtype=gb.GRB.BINARY, name='h')
h = model.addVars(N, N, T, range(K), vtype=gb.GRB.BINARY, name='h')

# Travel time traversing arc a at time t										(w)
# w[a,t] = travel time (continuous)
# w = model.addVars(len(A), len(T), lb=0, vtype=gb.GRB.CONTINUOUS, name='w')
w = model.addVars(A, T, lb=0, vtype=gb.GRB.CONTINUOUS, name='w')

# Travel time from origin to destination at time t								(v)
# v[o,d,t] = travel time (continuous)
# v = model.addVars(len(N), len(N), len(T), lb=0, vtype=gb.GRB.CONTINUOUS, name='v')
v = model.addVars(N, N, T, lb=0, vtype=gb.GRB.CONTINUOUS, name='v')

***

### Objective Function

In [5]:
# Minimize total passenger delays												(1)
model.setObjective(
	quicksum(
		quicksum(
			phi[o, d, t] * (v[o, d, t] - Omega[o, d]) for t in T
		) for o in N for d in N if o != d
	)
)

***

### Constraints

Maintainance scheduling constraints

In [6]:
# Job started and completed within time horizon									(2)
for j in J:
	model.addConstr(
		quicksum(y[j, t] for t in range(Tend - pi[j] + 1)) == 1
	)

# Availability of arc a at time t												(3)
# for a, (o, d) in enumerate(A):
for a in A:
	for t in T:
		# for j in Ja[(o, d)]:
		for j in Ja[a]:
			model.addConstr(
				x[a[0], a[1], t] + quicksum(y[j, tp] for tp in range(max(0, t-pi[j]+2), t+1)) <= 1
			)

# increased travel time for service replacement									(4)
# for a, (o, d) in enumerate(A):
for a in A:
	for t in T:
		model.addConstr(
			w[a[0], a[1], t] == x[a[0], a[1], t] * omega_e[a] + (1 - x[a[0], a[1], t]) * omega_j[a]
		)

# Ensure correct arc travel times for free variables							(5)
# for a, (o, d) in enumerate(A):
for a in A:
	model.addConstr(
		quicksum(x[a[0], a[1],t] for t in T) == len(T) - quicksum(pi[j] for j in Ja[a])
	)

Restrictions on maintenance scheduling

In [7]:
# Arcs that cannt be unavailable simultaneously									(6)
for t in T:
	for j in J:
		for c in C:
			model.addConstr(
				quicksum(1 - x[a[0], a[1], t] for a in c) <= 1
			)

# Non overlapping jobs on same arc												(7)
# for a, (o, d) in enumerate(A):
for a in A:
	for t in T:
		model.addConstr(
			quicksum(
				quicksum(
					y[j, tp] for tp in range(max(0, t-pi[j]-tau[a]+1), Tend)
				) for j in Ja[a]
			) <= 1
		)

Event Requests

In [8]:
# Each track segment in an event request has limited capacity					(8)

# TODO: This should be fixed... we should guarantee M is large enough (see inequality) 
# and also for now each s is a single arch but in principle it should be a set of arcs
# like [(1,2), (2,3), (3,4)] for example since it should model a track segment being under event
# --> after fixing these the extra sum term is needed

M = 1000 # Large potitive number for unilimited train capacity
for t in T:
	for a in Et[t]:
		model.addConstr(
			# quicksum(
			quicksum(
				quicksum(
					h[o, d, t, i]*beta[o, d, t]*phi[o, d, t] for i in range(K) if a in R[(o, d)][i]
				) for o in N for d in N if o != d
				# ) for a in s
			)
			<= gamma 
			# + M * quicksum(x[a[0], a[1], t] for a in s)
			+ M * x[a[0], a[1], t]
		)

Passenger Route Choice and Evaluation

In [9]:
# Passeger flow from o to d is served by one of predefined routes				(9)
for t in T:
	for o in N:
		for d in N:
			if o != d:
				model.addConstr(
					quicksum(
						h[o, d, t, i] for i in range(K)
					) == 1
				)

# Lower bound for travel time from o to d										(10)
for t in T:
	for o in N:
		for d in N:
			if o != d:
				for i in range(K):
					model.addConstr(
						v[o, d, t] >= quicksum(w[a[0], a[1], t] for a in R[(o, d)][i]) - M*(1 - h[o, d, t, i])
					)

# Upper bound for travel time from o to d										(11)
for t in T:
	for o in N:
		for d in N:
			if o != d:
				for i in range(K):
					model.addConstr(
						v[o, d, t] <= quicksum(w[a[0], a[1], t] for a in R[(o, d)][i])
					)

***

### Optimisation

In [10]:
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Arch Linux")

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 25677 rows, 17550 columns and 190647 nonzeros
Model fingerprint: 0xc79c7f5e
Variable types: 5700 continuous, 11850 integer (11850 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+03]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 21809 rows and 15019 columns
Presolve time: 0.28s
Presolved: 3868 rows, 2531 columns, 14822 nonzeros
Variable types: 0 continuous, 2531 integer (1627 binary)
Found heuristic solution: objective 3940854.0000
Found heuristic solution: objective 3939315.0000

Root relaxation: objective 3.905427e+06, 2282 iterations, 0.09 seconds (0.10 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl 